Transformações a serem realizadas

- [ ] Remoção de arquivos com horário de ping diferente do horário de requsição.
- [ ]  Ajustar hora em sp, diminuindo -3.
- [ ]  Em Curitiba, quando o campo “codigolinha” estiver “REC”, o ônibus não está em operação, logo, será removido.
- [ ] Ausência de valor no campo “linha” em BSB indica que não está em operação, logo deverá ser removido.
- [ ] Atualizar campos de horas e datas para ISO 8601  2024-02-24T13:05Z.
- [ ] Padronizar o sentido de operação da linha em SP e CWB para integers 1 = ida 2=  volta.
- [ ] Padronizar os identificadores de ônibus CUR_idOnibus.
- [ ] Add nome dos arquivos para um campo algo como "query_timestamp"


In [1]:
from pyspark.sql import *
from delta import *

builder = SparkSession.builder.appName("topicos").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"

print(spark.version)


24/03/11 15:59:03 WARN Utils: Your hostname, notebook resolves to a loopback address: 127.0.1.1; using 172.20.8.208 instead (on interface wlo1)
24/03/11 15:59:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/felipe/venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/felipe/.ivy2/cache
The jars for the packages stored in: /home/felipe/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-12409414-1093-4e0b-8fdc-cd853721e620;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.1.0/delta-spark_2.12-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.1.0!delta-spark_2.12.jar (1469ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.1.0/delta-storage-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.1.0!delta-storage.jar (274ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (314ms)
:: resolution report :: resolve 4520ms :

3.5.1


In [13]:
from pyspark.sql.functions import udf, input_file_name, col
from pyspark.sql.types import StringType
from datetime import datetime, timedelta
from pyspark.sql import functions as F

def changeBusIdSP(onibus_id):
    """
    Change the column "id_onibus" to the following pattern: CITY_id_onibus

    Example: SP_0881
    """
    return f"SPO_{onibus_id}"



def changeTimestamp(timestamp):
    """ 
    Change timestamp to ISO 8601 (2024-02-24T13:05Z) using GMT-3
    """
    datetime_object = datetime.fromisoformat(timestamp)
    return (datetime_object-timedelta(hours=3)).isoformat()

def addTimestampQueryTime(filenames):
    """ 
    Add the column "query_timestamp" indicating the timestamp
    
    file:///home/felipe/code/topicos_dados/dados/cb_micro/1706665101.9679544.parquet -> 1706665101.9679544
    """
    file_name = str(filenames)[54:-8]
    return str(datetime.fromtimestamp(float(file_name)))

def checkInvalidTimeRows(df):
    
    df = df.withColumn("tempo_captura", F.to_timestamp("tempo_captura"))
    df = df.withColumn("file_timestamp", F.to_timestamp("file_timestamp"))

    timediff = F.abs(F.unix_timestamp("tempo_captura") - F.unix_timestamp("file_timestamp"))

    return df.filter(timediff >= 300)


# UDFS

udf_transformBusIdSpo = udf(changeBusIdSP,StringType())
udf_changeTimestamp = udf(changeTimestamp,StringType())
udf_addTimestampFile = udf(addTimestampQueryTime,StringType())

# rj_df_busid = rj_delta.withColumn("bus_id", udf_transformBusIdRio(rj_delta["id_onibus"]))

sp = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro").withColumn("inputFiles",input_file_name())
sp = sp.withColumn("file_timestamp",udf_addTimestampFile(col("inputFiles")))
sp.select("file_timestamp").show()


+--------------------+
|      file_timestamp|
+--------------------+
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
|2024-01-30 22:39:...|
+--------------------+
only showing top 20 rows



In [3]:
from pyspark.sql import *
from delta import *
from pyspark.sql.functions import udf, input_file_name, col

builder = SparkSession.builder.appName("topicos") \
.config("spark.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
.config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog").config("spark.packages","io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.4").config("spark.hadoop.fs.s3a.access.key","DO00LNFAEQRLTA82CJCM").config("spark.hadoop.fs.s3a.secret.key","cOWHXzPEeesRVZSJDp/maav9CcnqYyyaOFh2HNWHcJ0").config("spark.hadoop.fs.s3a.endpoint", "nyc3.digitaloceanspaces.com")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"


sp = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro").withColumn("inputFiles",input_file_name())
# sp.write.mode("overwrite").format("delta").save("/home/felipe/code/topicos_dados/lake/bronze/sp/")
sp.write.mode("overwrite").format("delta").option("path","s3a://brbus-dataset/bronze/sp/").saveAsTable("bronze_sp2")

spark.stop()

24/03/04 14:03:55 WARN Utils: Your hostname, desktop resolves to a loopback address: 127.0.1.1; using 192.168.0.104 instead (on interface enp6s0)
24/03/04 14:03:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/felipe/.local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/felipe/.ivy2/cache
The jars for the packages stored in: /home/felipe/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e2c744be-da6c-4b8e-9e06-3f89623c3294;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 526ms :: artifacts dl 20ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

Py4JJavaError: An error occurred while calling o52.saveAsTable.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:767)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:710)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:672)
	at org.apache.spark.sql.delta.catalog.DeltaCatalog.$a

In [7]:
spark.table("bronze_sp2").show()

+-------------------+-------------------+--------------------+---------+-------+-------------------+--------------------+--------------------+
|           latitude|          longitude|       tempo_captura|id_onibus|      c|                lt0|                 lt1|          inputFiles|
+-------------------+-------------------+--------------------+---------+-------+-------------------+--------------------+--------------------+
|        -23.6492395|          -46.73245|2024-01-31T02:10:43Z|    71212|609F-10|TERM. PRINC. ISABEL|       CHÁC. SANTANA|file:///home/feli...|
|         -23.534974|        -46.6443435|2024-01-31T02:10:10Z|    71872|609F-10|TERM. PRINC. ISABEL|       CHÁC. SANTANA|file:///home/feli...|
|        -23.5825705|       -46.68577625|2024-01-31T02:10:21Z|    71740|609F-10|TERM. PRINC. ISABEL|       CHÁC. SANTANA|file:///home/feli...|
|       -23.55969475|         -46.654689|2024-01-31T02:10:51Z|    71754|609F-10|TERM. PRINC. ISABEL|       CHÁC. SANTANA|file:///home/feli...|

In [ ]:
s3://https://brbus-dataset.nyc3.digitaloceanspaces.com/bronze/sp/" 

In [5]:
from pyspark.sql import *
from delta import *
from pyspark.sql.functions import udf, input_file_name, col

builder = SparkSession.builder \
    .appName("Salvar Delta Table no Spaces") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.jars", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key", "DO00LNFAEQRLTA82CJCM") \
    .config("spark.hadoop.fs.s3a.secret.key", "cOWHXzPEeesRVZSJDp/maav9CcnqYyyaOFh2HNWHcJ0") \
    .config("spark.hadoop.fs.s3a.endpoint", "nyc3.digitaloceanspaces.com")

spark = builder.getOrCreate()


INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"


sp = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro").withColumn("inputFiles",input_file_name())
# sp.write.mode("overwrite").format("delta").save("/home/felipe/code/topicos_dados/lake/bronze/sp/")
sp.write.mode("overwrite").format("delta").option("path","s3a://brbus-dataset/bronze/sp/").saveAsTable("bronze_sp2")

spark.stop()

24/03/04 14:04:51 ERROR SparkContext: Error initializing SparkContext.
java.lang.IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: org.apache.hadoop:hadoop-aws:3.3.4
	at org.apache.hadoop.fs.Path.initialize(Path.java:263)
	at org.apache.hadoop.fs.Path.<init>(Path.java:221)
	at org.apache.spark.SparkContext.checkRemoteJarFile$1(SparkContext.scala:2116)
	at org.apache.spark.SparkContext.addJar(SparkContext.scala:2164)
	at org.apache.spark.SparkContext.$anonfun$new$15(SparkContext.scala:526)
	at org.apache.spark.SparkContext.$anonfun$new$15$adapted(SparkContext.scala:526)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:526)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jd

IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: org.apache.hadoop:hadoop-aws:3.3.4

In [3]:
from pyspark.sql import *
from delta import *
from pyspark.sql.functions import udf, input_file_name, col

builder = SparkSession.builder.appName("spark") \
.config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.config("spark.jars.packages","io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.4") \
.config("spark.hadoop.fs.s3a.access.key","DO00LNFAEQRLTA82CJCM") \
.config("spark.hadoop.fs.s3a.secret.key","cOWHXzPEeesRVZSJDp/maav9CcnqYyyaOFh2HNWHcJ0") \
.config("spark.hadoop.fs.s3a.endpoint", "nyc3.digitaloceanspaces.com") \

builder = SparkSession.builder.appName("spark").config("spark.hadoop.fs.s3a.access.key","DO00LNFAEQRLTA82CJCM").config("spark.hadoop.fs.s3a.secret.key","cOWHXzPEeesRVZSJDp/maav9CcnqYyyaOFh2HNWHcJ0").config("spark.hadoop.fs.s3a.endpoint", "nyc3.digitaloceanspaces.com")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"


sp = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro").withColumn("inputFiles",input_file_name())
# sp.write.mode("overwrite").format("delta").save("/home/felipe/code/topicos_dados/lake/bronze/sp/")
sp.write.mode("overwrite").format("delta").option("path","s3a://brbus-dataset/bronze/sp/").saveAsTable("bronze_sp2")

spark.stop()

24/03/04 14:44:42 ERROR Utils: Aborting task
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:767)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:710)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:672)
	at org.apache.spark.sql.delta.catalog.DeltaCatalog.$anonfun$createDeltaTab

Py4JJavaError: An error occurred while calling o84.saveAsTable.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:767)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:710)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:672)
	at org.apache.spark.sql.delta.catalog.DeltaCatalog.$a

In [2]:
from pyspark.sql import SparkSession
import os

# Set the path to the Hadoop AWS JAR
hadoop_aws_jar = "/home/felipe/hadoop-aws-3.3.4.jar"

# Set the environment variable to include the JAR in the classpath
os.environ["PYSPARK_SUBMIT_ARGS"] = f"--jars {hadoop_aws_jar} pyspark-shell"

# Create SparkSession
spark = SparkSession.builder \
    .appName("topicos") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.packages", "io.delta:delta-spark_2.12:3.1.0") \
    .config("spark.hadoop.fs.s3a.access.key", "DO00LNFAEQRLTA82CJCM") \
    .config("spark.hadoop.fs.s3a.secret.key", "cOWHXzPEeesRVZSJDp/maav9CcnqYyyaOFh2HNWHcJ0") \
    .config("spark.hadoop.fs.s3a.endpoint", "nyc3.digitaloceanspaces.com") \
    .getOrCreate()


INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"


sp = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro").withColumn("inputFiles",input_file_name())
# sp.write.mode("overwrite").format("delta").save("/home/felipe/code/topicos_dados/lake/bronze/sp/")
sp.write.mode("overwrite").format("delta").option("path","s3a://brbus-dataset/bronze/sp/").saveAsTable("bronze_sp2")

spark.stop()


NameError: name 'input_file_name' is not defined